In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document
markdown_path = "./README.md"
loader = UnstructuredMarkdownLoader(markdown_path)
data = loader.load()
assert len(data) == 1
assert isinstance(data[0], Document)
readme_content = data[0].page_content
print(readme_content[:250])

RAG_program


In [1]:
import sys
print(sys.path)

['/root/miniconda3/envs/ragenv/lib/python312.zip', '/root/miniconda3/envs/ragenv/lib/python3.12', '/root/miniconda3/envs/ragenv/lib/python3.12/lib-dynload', '', '/root/miniconda3/envs/ragenv/lib/python3.12/site-packages', '/root/RAG_program/src']
